## Scrape Data

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle

In [6]:
source_url = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

res = requests.get(source_url)
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
dfs = pd.read_html(str(table), header=0)
df = dfs[0]
# print(df[0].to_json(orient='records'))

## Data Cleaning

In [8]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
df = df.rename({'Neighbourhood': 'Neighborhood'}, axis=1)

In [12]:
df.Borough.value_counts()

Not assigned        77
Etobicoke           44
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

Separate Borough is valid and Borough = not assigned

In [13]:
df_borough = df[~(df['Borough'].str.lower() == 'not assigned')].copy()
df_no_borough = df[df['Borough'].str.lower() == 'not assigned'].copy()

print(df_borough.shape)
print(df_no_borough.shape)

(210, 3)
(77, 3)


If Neighbourhood is 'Not Assigned', set Neighbourhood = Borough

In [14]:
for idx, row in df_borough.iterrows():
    if row['Neighborhood'].lower() == 'not assigned':
        df_borough.at[idx, 'Neighborhood'] = df_borough.at[idx, 'Borough']


Aggregate the Neighbourhoods with comma

In [15]:
df_borough1 = df_borough.groupby(['Postcode', 'Borough']).agg(lambda s: ', '.join(s.unique())).reset_index()

In [16]:
df_borough1.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Combine the borough df and no borough df

In [17]:
df_result = df_borough1.append(df_no_borough)

In [18]:
df_result.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [19]:
with open(r'canada_postcodes_df.pkl', 'wb') as f:
    pickle.dump(df_result, f)

In [20]:
df_result.shape

(180, 3)